In [9]:
import os
import json
import pandas as pd
import numpy as np
from pandas import json_normalize
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
entries = os.listdir('/workspace/tu-feb-2023--mina-ad-mina/src/Deep_Learning/Milestone 3/AAPL')
entries

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['2015-02-10.json',
 '2016-01-24.json',
 '2014-08-10.json',
 '2014-11-05.json',
 '2015-11-29.json',
 '2014-01-04.json',
 '2016-01-08.json',
 '2015-05-31.json',
 '2016-01-05.json',
 '2014-02-11.json',
 '2016-01-31.json',
 '2015-05-21.json',
 '2015-03-21.json',
 '2016-01-07.json',
 '2015-01-09.json',
 '2014-10-04.json',
 '2014-01-07.json',
 '2015-02-24.json',
 '2014-01-02.json',
 '2015-01-18.json',
 '2015-10-16.json',
 '2015-12-25.json',
 '2014-06-24.json',
 '2014-10-29.json',
 '2014-05-04.json',
 '2014-02-04.json',
 '2014-05-10.json',
 '2016-01-21.json',
 '2015-09-01.json',
 '2014-06-03.json',
 '2016-03-21.json',
 '2015-03-19.json',
 '2014-02-16.json',
 '2015-06-06.json',
 '2014-12-27.json',
 '2015-09-07.json',
 '2015-09-09.json',
 '2014-04-14.json',
 '2014-09-01.json',
 '2015-10-30.json',
 '2015-09-10.json',
 '2015-05-06.json',
 '2014-12-21.json',
 '2016-03-01.json',
 '2016-03-12.json',
 '2015-03-27.json',
 '2015-06-10.json',
 '2015-11-17.json',
 '2014-06-09.json',
 '2015-01-01.json',


In [11]:
apple_tweets = pd.DataFrame()
all_dfs = []
for entry in entries:
    with open(f'/workspace/tu-feb-2023--mina-ad-mina/src/Deep_Learning/Milestone 3/AAPL/{entry}') as f:
        lines = f.readlines()
        data = [json_normalize(json.loads(line.split('\n')[0])) for line in lines]
        df1 = pd.concat(data)
        all_dfs.append(df1)
apple_tweets = pd.concat(all_dfs)

In [12]:
class PreProcessor:
    '''
    Easily performs all the standard preprocessing steps
    like removing stopwords, stemming, etc.
    Only input that you need to provide is the dataframe and column name for the tweets
    '''
    def __init__(self, df, column_name):
        self.data = df
        self.conversations = list(self.data[column_name])
        self.stopwords = set(stopwords.words('english'))
        self.stemmer = SnowballStemmer("english")
        self.preprocessed = []
        
    def tokenize(self, sentence):
        '''
        Splits up words and makes a list of all words in the tweet
        '''
        tokenized_sentence = word_tokenize(sentence)
        return tokenized_sentence
            
    def remove_stopwords(self, sentence):
        '''Removes stopwords like 'a', 'the', 'and', etc.'''
        filtered_sentence = []
        for w in sentence:
            if w not in self.stopwords and len(w) > 1 and w[:2] != '//' and w != 'https': 
                filtered_sentence.append(w)
        return filtered_sentence
    
    def stem(self, sentence):
        '''
        Stems certain words to their root form.
        For example, words like 'computer', 'computation'
        all get truncated to 'comput'
        '''
        return [self.stemmer.stem(word) for word in sentence]
    
    def join_to_string(self, sentence):
        '''
        Joins the tokenized words to one string.
        '''
        return ' '.join(sentence)
    
    def full_preprocess(self, n_rows=None):
        '''
        Preprocess a selected number of rows and
        connects them back to strings
        '''
        # If nothing is given do it for the whole dataset
        if n_rows == None:
            n_rows = len(self.data)
            
        # Perform preprocessing
        for i in range(n_rows):
            tweet = self.conversations[i]
            tokenized = self.tokenize(tweet)
            cleaned = self.remove_stopwords(tokenized)
            stemmed = self.stem(cleaned)
            joined = self.join_to_string(stemmed)
            self.preprocessed.append(joined)
        return self.preprocessed

In [13]:
# Preprocess text and put it in a new column
preprocessor = PreProcessor(apple_tweets, 'text')
apple_tweets['cleaned_text'] = preprocessor.full_preprocess()
apple_tweets.head()

/tmp/ipykernel_1563/3637825965.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  apple_tweets['cleaned_text'] = preprocessor.full_preprocess()


,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,retweeted_status.coordinates.coordinates,quoted_status.entities.trends,retweeted_status.quoted_status.entities.trends,quoted_status.entities.media,quoted_status.extended_entities.media,quoted_status.quoted_status_id,quoted_status.quoted_status_id_str,retweeted_status.scopes.followers,retweeted_status.place.contained_within,cleaned_text
0,Tue Feb 10 12:19:12 +0000 2015,565122812634214400,565122812634214400,RT @RiCHi: #iOS9 'Monarch' rumors: Stable and ...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rt richi ios9 monarch rumor stabl fast big fea...
0,Tue Feb 10 21:50:05 +0000 2015,565266480112357377,565266480112357377,RT @WSJ: Apple becomes the first U.S. company ...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rt wsj appl becom first u.s. compani close 700...
0,Tue Feb 10 18:26:00 +0000 2015,565215120855293952,565215120855293952,$AAPL Analyst: New Touch ID sensor in fall iPh...,"<a href=""http://twitterfeed.com"" rel=""nofollow...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aapl analyst new touch id sensor fall iphon lc...
0,Tue Feb 10 18:26:38 +0000 2015,565215280243027969,565215280243027969,"Welcome back, $AAPL 120+.\nDon't be a stranger...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,welcom back aapl 120+ do n't stranger feel fre...
0,Tue Feb 10 21:21:19 +0000 2015,565259240747851778,565259240747851778,RT @WSJ: Apple becomes the first U.S. company ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rt wsj appl becom first u.s. compani close 700...


In [14]:
apple_tweets['created_at'] = pd.to_datetime(apple_tweets['created_at'])
apple_tweets = apple_tweets[['created_at', 'text', 'cleaned_text']]
apple_tweets.head()

,created_at,text,cleaned_text
0,2015-02-10 12:19:12+00:00,RT @RiCHi: #iOS9 'Monarch' rumors: Stable and ...,rt richi ios9 monarch rumor stabl fast big fea...
0,2015-02-10 21:50:05+00:00,RT @WSJ: Apple becomes the first U.S. company ...,rt wsj appl becom first u.s. compani close 700...
0,2015-02-10 18:26:00+00:00,$AAPL Analyst: New Touch ID sensor in fall iPh...,aapl analyst new touch id sensor fall iphon lc...
0,2015-02-10 18:26:38+00:00,"Welcome back, $AAPL 120+.\nDon't be a stranger...",welcom back aapl 120+ do n't stranger feel fre...
0,2015-02-10 21:21:19+00:00,RT @WSJ: Apple becomes the first U.S. company ...,rt wsj appl becom first u.s. compani close 700...


In [15]:
df = df.reset_index()
df.head()

NameError: name 'df' is not defined

In [ ]:
# # Convert the 'created_at' column in the tweets dataframe to datetime format
apple_tweets['created_at'] = pd.to_datetime(apple_tweets['created_at'], utc=True).dt.date

# # Convert the 'Date' column in the stocks dataframe to datetime format
df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.date
merged_df = pd.merge(apple_tweets, df, left_on='created_at', right_on='Date')
# merged_df['price_change'] = merged_df['Close'] - merged_df['Open']
# merged_df.drop('Date', axis=1, inplace=True)
merged_df.head()

In [ ]:
merged_df['price_change'] = merged_df['Close'] - merged_df['Open']
merged_df.drop('Date', axis=1, inplace=True)
merged_df.head()

In [ ]:
merged_df.drop(columns=['High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
merged_df.head()

In [ ]:
training_set_size = int(len(df)*.8)
training_set = merged_df.iloc[:training_set_size,-1:].values
test_set = merged_df.iloc[training_set_size:,-1:].values

In [ ]:
import nltk
nltk.download('vader_lexicon')  # download the pre-built sentiment lexicon

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
scores = []
for ind in merged_df.index:
    scores.append(analyzer.polarity_scores(merged_df['cleaned_text'][ind])['compound'])

In [ ]:
merged_df['sentiment_scores'] = scores
merged_df.head()

In [ ]:
merged_df['price_increase'] = merged_df['price_change'] > 0
merged_df['pred_price_increase'] = merged_df['sentiment_scores'] >= 0
merged_df.head(10)

In [ ]:
grouped_df = merged_df.groupby('created_at').agg({'price_change': 'first', 'sentiment_scores': 'sum'}).reset_index()
grouped_df['price_increase'] = grouped_df['price_change']>0
grouped_df['pred_price_increase'] = grouped_df['sentiment_scores']>0
grouped_df.head()

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
display = PrecisionRecallDisplay.from_predictions(grouped_df['price_increase'], grouped_df['pred_price_increase'], name="LinearSVC")

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification

fpr, tpr, threshold = metrics.roc_curve(grouped_df['price_increase'], grouped_df['pred_price_increase'])
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
# AUC ROC Plot
plt.title('AUC ROC Plot')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# Accuracy Plot
f1 = plt.figure()
plt.title('Accuracy')
plt.plot(grouped_df['price_increase'], 'b', label = 'Actual')
plt.plot(grouped_df['pred_price_increase'], 'r', label = 'Prediction')

plt.legend(loc = 'lower right')
plt.show()